In [1]:
from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib import rnn

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/mnist/", one_hot=True)

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


To classify images using RNN we consider every image as sequence of data. As MNIST images are 28X28 pixel image, each 
image row is considered as 28 seqeunce of 28 steps.

In [3]:
learning_rate = 0.001
nb_epoch = 10
batch_size = 64
display_step = 2

In [4]:
n_input = 28 # MNIST data input
n_steps = 28 # time step
n_hidden = 128 #Hidden layer num of features
n_classes = 10 # Number of digit classes

In [5]:
X = tf.placeholder('float', [None, n_steps, n_input])
y = tf.placeholder('float', [None, n_classes])

In [6]:
weights = {
    'out' : tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases  = {
    'out' : tf.Variable(tf.random_normal([n_classes]))
}

In [9]:
def RNN(X, weights, biases):
    #Prepare data for the RNN function requirement
    #Current shape is [batch_size X n_steps X n_input]
    #Required shape is 'n_steps' list of tensor of shape = [batch_size X n_input]
    #permuting batch_size and n_steps
    X = tf.transpose(X, [1, 0, 2])
    #Reshaping [(n_steps * batch_size), n_input]
    X = tf.reshape(X, [-1, n_input])
    #split to get n_steps tensors of shape batch_size X n_input
    X = tf.split(0, n_steps, X)
    
    #define a lstm cell using tensorflow
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    
    #get a lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, X, dtype=tf.float32)

    #Linear activation - using rnn innner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [10]:
pred = RNN(X, weights, biases)
#define the loss and the optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptmizer(learning_rate=learning_rate).minimize(loss)
#Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

AttributeError: 'module' object has no attribute 'BasicLSTMCell'

In [ ]:
#launch the graph
# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={X: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={X: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={X: batch_x, y: batch_y})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label}))